In [ ]:
# 데이터 로드
from google.colab import drive

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

import pandas as pd

# 데이터셋 분할
from sklearn.model_selection import train_test_split

# 시퀀스 신경망, 은닉층, 드롭아웃 신경망
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# 정규화 패키지
from sklearn.preprocessing import MinMaxScaler

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### 함수 정의 구간
# 데이터 업로드 _utf
def upload_data_utf(file_name):
  rtn = pd.read_csv(file_name, encoding = 'utf-8')
  rtn = pd.DataFrame(rtn)

  return rtn

def upload_data_cp(file_name):
  rtn = pd.read_csv(file_name, encoding = 'cp949', delimiter = '\t')
  rtn = pd.DataFrame(rtn)

  return rtn

### 데이터 정규화
  # Min-Max Scaling
  # 0에서 1사이로 정규화
def scaling(data, loctaion):

  # 스케일 대상 column
  scaleColumn = ['건축년도', '전용면적'] + location
  rtn = data

  # 데이터 정규화
  scaler = MinMaxScaler()
  rtn[scaleColumn] = scaler.fit_transform(rtn[scaleColumn])

  return rtn

def create_model(train_data, dropout_rate):
  # 다층 퍼셉트론(MLP) 모델 생성
  model = Sequential()
  model.add(Dense(128, activation='relu', input_shape=(train_data.shape[1],)))  # 첫 번째 은닉층
  model.add(Dropout(dropout_rate))
  model.add(Dense(64, activation='relu'))  # 두 번째 은닉층
  model.add(Dense(1))  # 출력층

  return model

In [ ]:
# 안양 XDATA 파일 업로드
AnyangData = upload_data_utf("/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/AnyangData/XDATA.csv")
AnyangY = AnyangData[['거래금액', '거래일자']]

# 수원 XDATA 파일 업로드
SuwonData = upload_data_utf("/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/SuwonData/XDATA.csv")
SuwonY = SuwonData[['거래금액', '거래일자']]

In [ ]:
# 입지 정보 칼럼명
location = ['약국', '의원', '병원', '대병원', '대형상권', '공원', '도서관', '유치원', '초등학교', '중학교', '고등학교', '학원', '지하철']

AnyangX = AnyangData[['아파트', '건축년도', '전용면적', '위도', '경도'] + location]
SuwonX = SuwonData[['아파트', '건축년도', '전용면적', '위도', '경도'] + location]

In [ ]:
# 데이터 정규화
x_data = scaling(x_data, location)
x_data = x_data[['건축년도', '전용면적'] + location]

# 훈련 세트와 테스트 세트로 데이터 나누기
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [ ]:
model = create_model(x_train, 0.2)
model.compile(optimizer = 'adam', loss = 'mse')

# 모델 학습
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_test, y_test))

Epoch 1/100
92/92 [==============================] - 1s 5ms/step - loss: 3938479616.0000 - val_loss: 3916272384.0000
Epoch 2/100
92/92 [==============================] - 0s 3ms/step - loss: 3899361280.0000 - val_loss: 3831663616.0000
Epoch 3/100
92/92 [==============================] - 0s 3ms/step - loss: 3741117696.0000 - val_loss: 3582617600.0000
Epoch 4/100
92/92 [==============================] - 0s 3ms/step - loss: 3386893824.0000 - val_loss: 3113914112.0000
Epoch 5/100
92/92 [==============================] - 0s 3ms/step - loss: 2818726912.0000 - val_loss: 2456302848.0000
Epoch 6/100
92/92 [==============================] - 0s 3ms/step - loss: 2116256768.0000 - val_loss: 1734547072.0000
Epoch 7/100
92/92 [==============================] - 0s 2ms/step - loss: 1440067968.0000 - val_loss: 1142321152.0000
Epoch 8/100
92/92 [==============================] - 0s 3ms/step - loss: 958739072.0000 - val_loss: 793294272.0000
Epoch 9/100
92/92 [==============================] - 0s 3ms/step -

In [ ]:
predictions = model.predict(x_test)
print(predictions)

mse = mean_squared_error(y_test, predictions)
print(mse)

r2 = r2_score(y_test, predictions)
print(r2)